# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kandiari,26.9156,68.5216,34.94,35,2,4.30,PK,1694313093
1,1,vilyuysk,63.7553,121.6247,2.82,92,100,2.35,RU,1694313093
2,2,albany,42.6001,-73.9662,22.04,89,100,1.34,US,1694313020
3,3,tiksi,71.6872,128.8694,3.73,66,28,2.46,RU,1694313038
4,4,plavsk,53.7094,37.2919,12.65,78,100,3.81,RU,1694313094


In [8]:
!pip install cartopy
!pip install geoviews
!pip install pyproj

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,tagiura,32.8817,13.3506,25.32,69,0,4.25,LY,1694313109
76,76,colonia,50.9333,6.9500,21.09,91,0,1.03,DE,1694312999
85,85,tazacorte,28.6290,-17.9293,22.54,80,0,4.02,ES,1694313128
105,105,russellville,35.2784,-93.1338,23.32,59,0,3.60,US,1694313136
112,112,tabas,33.5959,56.9244,25.12,18,0,1.03,IR,1694313139
136,136,carnarvon,-24.8667,113.6333,21.86,61,0,0.99,AU,1694313148
179,179,palmer,42.1584,-72.3287,21.46,85,0,2.06,US,1694313187
258,258,east end,34.5507,-92.3410,23.84,70,0,3.15,US,1694313213
275,275,solnechnyy,50.7214,136.6319,21.20,34,0,2.46,RU,1694312964
282,282,birzebbuga,35.8258,14.5269,24.53,88,0,2.06,MT,1694313223


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
39,tagiura,LY,32.8817,13.3506,69,
76,colonia,DE,50.9333,6.9500,91,
85,tazacorte,ES,28.6290,-17.9293,80,
105,russellville,US,35.2784,-93.1338,59,
112,tabas,IR,33.5959,56.9244,18,
136,carnarvon,AU,-24.8667,113.6333,61,
179,palmer,US,42.1584,-72.3287,85,
258,east end,US,34.5507,-92.3410,70,
275,solnechnyy,RU,50.7214,136.6319,34,
282,birzebbuga,MT,35.8258,14.5269,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tagiura - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
tazacorte - nearest hotel: App Leyma
russellville - nearest hotel: Holiday Inn
tabas - nearest hotel: هتل امیر
carnarvon - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
east end - nearest hotel: No hotel found
solnechnyy - nearest hotel: Заря
birzebbuga - nearest hotel: Seabreeze
belmonte - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
shiguai - nearest hotel: No hotel found
huntsville - nearest hotel: 106 Jefferson Huntsville, Curio Collection by Hilton
chifeng - nearest hotel: 心连心酒店
rosetta - nearest hotel: No hotel found
lamar - nearest hotel: 1st St Lofts
kill devil hills - nearest hotel: Mariner Days Inn & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
39,tagiura,LY,32.8817,13.3506,69,No hotel found
76,colonia,DE,50.9333,6.9500,91,Wasserturm Hotel Cologne
85,tazacorte,ES,28.6290,-17.9293,80,App Leyma
105,russellville,US,35.2784,-93.1338,59,Holiday Inn
112,tabas,IR,33.5959,56.9244,18,هتل امیر
136,carnarvon,AU,-24.8667,113.6333,61,No hotel found
179,palmer,US,42.1584,-72.3287,85,No hotel found
258,east end,US,34.5507,-92.3410,70,No hotel found
275,solnechnyy,RU,50.7214,136.6319,34,Заря
282,birzebbuga,MT,35.8258,14.5269,88,Seabreeze


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)